In [ ]:
#| default_exp interface_pymeshlab

## `pymeshlab` interface

> Convert `ObjMesh` to `pymeshlab`'s mesh class to apply meshlab filters.

In this notebook, we define functions to convert our `ObjMesh` class to and from `pymeshlab`.

In [ ]:
#| export

from blender_tissue_cartography import mesh as tcmesh

import numpy as np
import pymeshlab
import warnings

Unable to load the following plugins:

	libio_e57.so: libio_e57.so does not seem to be a Qt Plugin.

Cannot load library /home/nikolas/Programs/miniconda3/envs/blender-tissue-cartography/lib/python3.11/site-packages/pymeshlab/lib/plugins/libio_e57.so: (/lib/x86_64-linux-gnu/libp11-kit.so.0: undefined symbol: ffi_type_pointer, version LIBFFI_BASE_7.0)



In [ ]:
#| hide
np.set_printoptions(suppress=True)

In [ ]:
#| export

def convert_to_pymeshlab(mesh: tcmesh.ObjMesh, add_texture_info=None) -> pymeshlab.Mesh:
    """
    Convert tcmesh.ObjMesh to pymeshlab.Mesh.
    
    See https://pymeshlab.readthedocs.io/en/latest/classes/mesh.html
    Note: normal information is recalculated by pymeshlab. Discards any non-triangle faces.
    
    The texture information is saved as a vertex attribute via v_tex_coords_matrix. Note that this discards
    information since a vertex can have multiple texture coordinates. For this reason,
    we also save it as wedge_tex_coord_matrix (i.e. per triangle).
    
    Parameters
    ----------
    mesh : tcmesh.ObjMesh
    add_texture_info : None or bool
        Whether to add texture info to the pymeshlab.Mesh. If None, texture is added if
        available for at least one vertex.
    Returns
    -------
    pymeshlab.Mesh

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular. discarding non-triangular faces")
    add_texture_info = ((not mesh.only_vertices and len(mesh.texture_vertices) > 0)
                        if add_texture_info is None else add_texture_info)
    if mesh.tris.shape[0] == 0:
        return pymeshlab.Mesh(vertex_matrix=mesh.vertices)
    if not add_texture_info:
        return pymeshlab.Mesh(vertex_matrix=mesh.vertices, face_matrix=mesh.tris)
    wedge_coords = mesh.texture_vertices[mesh.texture_tris].reshape((-1,2), order="C")
    converted = pymeshlab.Mesh(vertex_matrix=mesh.vertices, face_matrix=mesh.tris,
                               v_tex_coords_matrix=mesh.vertex_textures,
                               w_tex_coords_matrix=wedge_coords)
    return converted

In [ ]:
mesh_fname_data = "registration_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj"
mesh_fname_ref = "registration_example/Drosophila_reference.obj"

mesh_data = tcmesh.ObjMesh.read_obj(mesh_fname_data)
mesh_ref = tcmesh.ObjMesh.read_obj(mesh_fname_ref)

  o embryo_rect


In [ ]:
pymesh_data = convert_to_pymeshlab(mesh_data)

In [ ]:
pymesh_normals = pymesh_data.vertex_normal_matrix()
pymesh_normals = (pymesh_normals.T / np.linalg.norm(pymesh_normals, axis=-1)).T

mesh_normals = (mesh_data.normals.T / np.linalg.norm(mesh_data.normals, axis=-1)).T

np.einsum('vi,vi->v', mesh_normals, pymesh_normals)

array([0.99999487, 0.99997637, 0.99996542, ..., 0.99966711, 0.99984559,
       0.99975186])

In [ ]:
np.allclose(mesh_data.vertices, pymesh_data.vertex_matrix())

True

In [ ]:
### check correctness of wedge coords

mesh = tcmesh.ObjMesh.read_obj("movie_example/mesh_subdiv.obj")
converted = convert_to_pymeshlab(mesh)

In [ ]:
ms = pymeshlab.MeshSet()
#ms.add_mesh(converted)
ms.load_new_mesh("movie_example/mesh_subdiv.obj")

In [ ]:
np.allclose(np.nan_to_num(ms.current_mesh().wedge_tex_coord_matrix()),
            np.nan_to_num(converted.wedge_tex_coord_matrix()))

True

### Transforming back to `ObjMesh`

In [ ]:
#| export

def convert_from_pymeshlab(mesh: pymeshlab.Mesh, reconstruct_texture_from_faces=True,
                           texture_vertex_decimals=10) -> tcmesh.ObjMesh:
    """
    Convert pymeshlab mesh to ObjMesh.
    
    Texture vertices can be reconstructed from wedge_tex_coord_matrix (per face) or from the
    vertex attribute vertex_tex_coord_matrix. Reconstruction from face texture can accommodate
    multiple texture coordinates per vertex (e.g. for UV maps with seams).
    
    Texture vertices are rounded to texture_vertex_decimals decimals
    """
    vertices = mesh.vertex_matrix()
    normals = mesh.vertex_normal_matrix()
    normals = (normals.T / np.linalg.norm(normals, axis=-1)).T
    if mesh.face_matrix().shape[0] == 0:
        return tcmesh.ObjMesh(vertices=vertices, faces=[])
    if not mesh.has_vertex_tex_coord():
        faces = mesh.face_matrix()
        return tcmesh.ObjMesh(vertices=vertices, faces=faces, normals=normals)
    if mesh.has_vertex_tex_coord() and not reconstruct_texture_from_faces:
        faces = [[2*[v,] for v in f] for f in mesh.face_matrix()]
        return tcmesh.ObjMesh(vertices=vertices, faces=faces, normals=normals,
                            texture_vertices=mesh.vertex_tex_coord_matrix())
    # reconstruct texture vertices - big pain.
    wegde_coords = mesh.wedge_tex_coord_matrix()
    wegde_coords = wegde_coords.reshape((-1,3,2), order="C").reshape((-1,2), order="F")
    wegde_coords = np.round(wegde_coords, decimals=texture_vertex_decimals)
    texture_vertices_unique, index, inverse_index = np.unique(wegde_coords, axis=0,
                                                              return_index=True, return_inverse=True)
    sort_index = index.argsort()
    reorder = {x : i for i, x in enumerate(sort_index)}
    texture_vertices_unique = texture_vertices_unique[sort_index]
    n_faces = mesh.face_matrix().shape[0]
    faces = [[[v, reorder[inverse_index[ifc+iv*n_faces]]] for iv, v in enumerate(fc)]
             for ifc, fc in enumerate(mesh.face_matrix())]

    return tcmesh.ObjMesh(vertices=vertices, faces=faces, normals=normals, texture_vertices=texture_vertices_unique)

In [ ]:
pymesh_ref = convert_to_pymeshlab(mesh_ref)
pymesh_ref_converted = convert_from_pymeshlab(pymesh_ref)

In [ ]:
np.allclose(mesh_ref.texture_vertices[mesh_ref.texture_tris],
 pymesh_ref_converted.texture_vertices[pymesh_ref_converted.texture_tris])

True

In [ ]:
mesh_seams = tcmesh.ObjMesh.read_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv.obj")
pymesh_seams = convert_to_pymeshlab(mesh_seams,add_texture_info=True)
mesh_seams_reconverted = convert_from_pymeshlab(pymesh_seams, reconstruct_texture_from_faces=True)
mesh_seams_reconverted.write_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv_resaved.obj")

  o Drosophila_CAAX-mCherry_mesh_remeshed
  l 2534 8160
/tmp/ipykernel_1942205/15752383.py:16: RuntimeWarning: invalid value encountered in divide
  normals = (normals.T / np.linalg.norm(normals, axis=-1)).T


In [ ]:
mesh_seams.texture_vertices.shape, mesh_seams_reconverted.texture_vertices.shape

((8288, 2), (8288, 2))

You can now use `MeshLab` filters like in the following, and the face attributes will hopefully be correctly updated.

In [ ]:
ms = pymeshlab.MeshSet()
ms.add_mesh(pymesh_seams)

ms.meshing_merge_close_vertices(threshold=pymeshlab.PercentageValue(100))
pymesh_seams_remeshed = ms.current_mesh()

convert_from_pymeshlab(pymesh_seams_remeshed, reconstruct_texture_from_faces=True,
                       texture_vertex_decimals=12)

<blender_tissue_cartography.mesh.ObjMesh>